<center><h1> Assignment 0 </h1></center>

This assignment is designed to confirm your background knowledge in the fundamentals of Linear Algebra, Probability, and Fully-connected Neural Networks. All the imports made in this notebook are as below; if these imports work, you are (mostly) set to complete the assignment. You are encouraged to read/view optional materials as needed, and some information will be provided in the upcoming programming review (Friday).

In [ ]:
import numpy as np
import requests
import io
import math

import torch
import torch.nn.functional as f

<table>
  <tr>
    <td>**Section**</td>
    <td>**Score**</td>
  </tr>
  <tr>
    <td> Multivariate Gaussians <br> [Q1 + Q2 + Q3] </td>
    <td> 3 + 3 + 4 </td>
  </tr>
  <tr>
    <td> Invertible Neural Network <br> [forward + invert + bonus] </td>
    <td> 2 + 5 + 5</td>
  </tr>
    <tr>
    <td> SGD </td>
    <td> 3 </td>
  </tr>
  <tr>
    <td> **Total**  </td>
    <td> **20** <br>[+5 bonus] </td>
  </tr>
  </table>

# I. Multivariate Gaussians

In the following, we will refer to the matrix $\mathbf{A}_{M\times M}$ and the column vectors $\mathbf{y}_{M\times 1}$, $\mathbf{x}_{M\times 1}$ and $\mu_{M\times 1}$.


[Q1] If $\mathbf{y} = \mathbf{A}\mathbf{x} + \mu$ and $\mathbf{x} \sim N(0,I_M)$ show that:

1. $\mathbb{E}[\mathbf{y}]=\mu$ 
2. $\mathbb{E}[\left(\mathbf{y-\mu}\right)\left(\mathbf{y-\mu}\right)^\top]=\mathbf{A}\mathbf{A}^\top$.

[Q2] Fill out the following functions for confirming the above. 

Ans. 1

1. For x~N(0,I) i.e x=(x1,x2,....xn)T  where xi are N(0,1) random variables.then E(x)=(E(x1),,,,E(xn))T = (0,,,,,,0)T => E[x]=0 

  E[y]= E[Ax+μ]= μ +AE[x]= μ

2. For x~N(0,I)  then E[(xi-E[xi])(xj-E[xj])]=E[xi.xj]= 1 (if i=j) & 0 (if i!=j)   

                      ==> E[x.(x)T]=I

   E[(y-μ).(y-μ)T]= E[(Ax).(Ax)T]= AE[x.(x)T](A)T = A.I.(A)T= A(A)T

In [ ]:
# Fill out these functions

def E_y(ys:np.array):
    """
    ys: A batch of column vectors
    return: Mean of the column vectors
    """
    return np.mean(ys,axis=0)

def E_yyT(ys:np.array):
    
    """
    ys: A batch of column vectors
    return: Covariance matrix of the column vectors
    """
    M=ys.shape[1]
    ys=np.reshape(ys,(50000,M));
    return np.cov(ys,rowvar=False)

In [ ]:
# Code to test the above two functions. 

M=2

# Observe how the matrix shapes are adjusted to allow for the vectorization.
A = np.random.randint(low=1,high=10,size=(1,M,M))
xs = np.random.randn(50000,M,1)
mu = np.random.randint(low=0,high=10,size=(1,M,1))

ys = A @ xs + mu
# The following two column-vectors should be nearly identical
print("Mean:")
print(E_y(ys))
print(mu,end="\n\n")


# The following two matrices should be nearly identical
print("Covariance:")
print(E_yyT(ys-mu))
print(A[0]@A[0].T)


Mean:
[[5.99705056]
 [0.99194652]]
[[[6]
  [1]]]

Covariance:
[[65.01380163 78.98194523]
 [78.98194523 96.92930433]]
[[65 79]
 [79 97]]


[Q3] Given a covariance matrix $\mathbf{C}$ and mean $\mu$, generate N samples of $y$ using $\mathbf{x} \sim N(0, I_m)$ such that $\mathbb{E}[\mathbf{y}]=\mu$ 
and $\mathbb{E}[\left(\mathbf{y-\mu}\right)\left(\mathbf{y-\mu}\right)^\top]=\mathbf{C}$.

Hint: See np.linalg.cholesky

In [ ]:
def sample(C, mu, N):
    """
    C: An MxM "positive definite"(?!?!) symmetric matrix
    mu: An Mx1 column vector 
    return: A batch of column-vectors with Mean=mu and Covariance=C
    """
    M = C.shape[0]
    # Write code here
    L = np.expand_dims(np.linalg.cholesky(C),axis=0)

    u = np.random.rand(M*N)
    v = np.random.rand(M*N) 
    func = np.vectorize(normal)
    u = func(u,v) 
    u = u.reshape(N,M)

    #u = np.random.normal(loc=0, scale=1, size=M*N).reshape(N,M)
    u = np.expand_dims(u,axis=2)
    y = np.matmul(L,u)+mu

    return y

def normal(u,v,mean=0,std=1):
  #Box Muller
  u1= 2.0 * math.pi * u
  v1 = math.sqrt( -2.0*math.log(1.0 - v))
  return v1 * math.sin(u1)

In [ ]:
# Code to test the above function 

M = 5

# The following ensures that C is a positive semi-definite symmetric matrix. Find out why! What happens if low=0?
A = np.random.randint(low=1,high=10,size=(M,M))
A[np.triu_indices(M,k=1)] = 0

C = A@A.T

mu = np.random.randint(low=0,high=10,size=(1,M,1))

ys = sample(C, mu, 50000)
# The following two column-vectors should be nearly identical
print("Mean:")
print(E_y(ys))
print(mu,end="\n\n")

# The following two matrices should be nearly identical
print("Covariance:")
print(E_yyT(ys-mu))
print(C)

Mean:
[[8.03183009]
 [1.02378772]
 [3.99629589]
 [0.94745071]
 [2.98847789]]
[[[8]
  [1]
  [4]
  [1]
  [3]]]

Covariance:
[[ 64.18567273  56.46962725   8.12559536  16.1292677   64.38971797]
 [ 56.46962725  74.68895562  17.14565489  38.91244938  86.38882663]
 [  8.12559536  17.14565489   6.0363125   20.05871838  22.08989346]
 [ 16.1292677   38.91244938  20.05871838 175.2206143  134.43632733]
 [ 64.38971797  86.38882663  22.08989346 134.43632733 172.51932642]]
[[ 64  56   8  16  64]
 [ 56  74  17  39  86]
 [  8  17   6  20  22]
 [ 16  39  20 174 134]
 [ 64  86  22 134 172]]


# II. Invertible Neural Network

Invertible neural networks -- which learn bijective (invertable) functions $f$ such that if $y=f(x)$, we should be  able to obtain an unique $x=f^{-1}(y)$ -- play a central role in many applications. In this assignment, we will work with invertible fully-connected 3-Layer Neural Networks. 

Recall that the main operation in neural networks is matrix multiplication and in order to compute the inverse, we would need to invert the matrices. As computing inverse is costly, we store each matrix in their SVD factorized forms $W = USV$, where U and V are orthogonal matrices and S is diagonal [with none of the diagonal entries being 0]. Also, observe that the number of hidden units at all layers are equal to the number of input units (why?!).

In this assignment, we will use parameters of an "already-invertible" neural network and just write the code for computing the forward and inversion operations. 

Write the code for forward pass, by choosing "LeakyReLU" as the activation function with negative_slope=0.1. Subsequently, write the code for the invert function.

In [ ]:
import torch.nn.functional as F

class InvertibleNN(torch.nn.Module):
    def __init__(self, m):
        
        super(InvertibleNN,self).__init__()
                
        self.u1 = torch.nn.Parameter(torch.nn.init.normal_(torch.empty(m,m)))
        self.s1 = torch.nn.Parameter(torch.nn.init.normal_(torch.empty(m,m)))
        self.v1 = torch.nn.Parameter(torch.nn.init.normal_(torch.empty(m,m)))
        self.b1 = torch.nn.Parameter(torch.nn.init.normal_(torch.empty(m)))
        
        self.u2 = torch.nn.Parameter(torch.nn.init.normal_(torch.empty(m,m)))
        self.s2 = torch.nn.Parameter(torch.nn.init.normal_(torch.empty(m,m)))
        self.v2 = torch.nn.Parameter(torch.nn.init.normal_(torch.empty(m,m)))
        self.b2 = torch.nn.Parameter(torch.nn.init.normal_(torch.empty(m)))
        
        self.u3 = torch.nn.Parameter(torch.nn.init.normal_(torch.empty(m,m)))
        self.s3 = torch.nn.Parameter(torch.nn.init.normal_(torch.empty(m,m)))
        self.v3 = torch.nn.Parameter(torch.nn.init.normal_(torch.empty(m,m)))
        self.b3 = torch.nn.Parameter(torch.nn.init.normal_(torch.empty(m)))
    
    def forward(self, x):
        """
        x: A batch input of shape (N,m)
        return: The output of the fully-connected 3-layer neural network
        """
        # Write code here
        self.w1=self.u1 @ torch.diag(torch.reciprocal(self.s1)) @ torch.t(self.v1)
        self.w2=self.u2 @ torch.diag(torch.reciprocal(self.s2)) @ torch.t(self.v2)
        self.w3=self.u3 @ torch.diag(torch.reciprocal(self.s3)) @ torch.t(self.v3)
        self.a1= torch.matmul(x,self.w1)+self.b1
        self.a1 = F.leaky_relu(self.a1,negative_slope=0.1)
        self.a2 = torch.matmul(self.a1,self.w2)+self.b2
        self.a2 = F.leaky_relu(self.a2,negative_slope=0.1)
        self.a3 = torch.matmul(self.a2,self.w3)+self.b3
        self.y=F.leaky_relu(self.a3,negative_slope=0.1)
        return self.y
    
    def invert(self, y):
        """
        y: A batch input of shape (N,m)
        return: The input of the fully-connected 3-layer neural network which would yield y
        """
        
        # Write code here
        lrate = 0.1

        y = y.reshape(-1,y.shape[0])
        y *= lrelu_prime(self.y)
        #w3_p = torch.matmul(torch.t(self.a2),y)
        #b3_p = y.clone() 
        y3 = torch.matmul(y,torch.t(self.w3)) - self.b3
        #y3 = y3.reshape(-1,1)
        
        y3 *= lrelu_prime(self.a2)
        #w2_p = torch.matmul(torch.t(self.a1),y3)
        #b2_p = y3.clone()
        y2= torch.matmul(y3,torch.t(self.w2)) - self.b2
        #y2 = y2.reshape(-1,1)
        
      
        y2 *= lrelu_prime(self.a1)
        #w1_p = torch.matmul(torch.t(x),y2)
        #b1_p = y2.clone()
        #d = x.squeeze(0)
        y1 = torch.matmul(y2,torch.t(self.w1)) - self.b1

        self.w3 += torch.matmul(torch.t(self.a2),y3) * lrate
        self.w2 += torch.matmul(torch.t(self.a1),y2) * lrate
        self.w1 += torch.matmul(x,y1) * lrate
        #self.b3 += y3.sum() * lrate
        #self.b2 += y2.sum() * lrate
        #self.b1 += y1.sum() * lrate
        return y1

  
def lrelu_prime(x,a=0.1):
  return torch.where(x >= 0, torch.tensor(1.), torch.tensor(a))

        
        

In [ ]:
# Code to test the forward and invert functions


for m in range(1,11):
    iNN = InvertibleNN(m)
    
    r = requests.get("https://web.cs.dal.ca/~sastry/inv_net_{}.pt".format(m))
    iNN.load_state_dict(torch.load(io.BytesIO(r.content)))

    x = torch.FloatTensor(np.random.randint(low=1,high=10,size=(1,m)))
    # These 2 outputs should be nearly identical. 
    print(x)
    print(iNN.invert(iNN(x)))
    
    print("="*40)

tensor([[7.]])
tensor([-2.2257], grad_fn=<SubBackward0>)
tensor([[3., 2.]])
tensor([-6.9207, -6.9207], grad_fn=<SubBackward0>)
tensor([[7., 7., 6.]])
tensor([ -7.2828, -10.2828,  -9.2828], grad_fn=<SubBackward0>)
tensor([[7., 8., 2., 1.]])
tensor([-6.0015, -5.0015, -7.0015, -4.0015], grad_fn=<SubBackward0>)
tensor([[4., 9., 4., 8., 9.]])
tensor([-3.4592, -4.4592, -4.4592, -5.4592, -4.4592], grad_fn=<SubBackward0>)
tensor([[2., 3., 7., 4., 2., 1.]])
tensor([ 8.9218, 10.9218,  7.9218, 13.9218,  7.9218,  6.9218],
       grad_fn=<SubBackward0>)
tensor([[7., 1., 2., 8., 1., 5., 1.]])
tensor([-1.8621,  2.1379, -2.8621, -0.8621, -3.8621, -5.8621,  1.1379],
       grad_fn=<SubBackward0>)
tensor([[8., 8., 8., 3., 9., 8., 4., 6.]])
tensor([-4.5493, -0.5493, -3.5493,  1.4507, -0.5493, -2.5493, -5.5493, -1.5493],
       grad_fn=<SubBackward0>)
tensor([[8., 7., 8., 5., 5., 3., 6., 3., 6.]])
tensor([-0.2130, -2.2130, -3.2130, -1.2130,  1.7870, -2.2130, -1.2130, -3.2130,
        -3.2130], grad_fn=<Su

[Bonus] In certain applications, it is important to calculate the trace of the input-output Jacobian matrix. Shown below is the code for computing the trace of the input-output jacobian. In general, for an N-dimensional input, we would require N `backward` passes [which can easily blow up in real-world settings]. However, we can reduce the number of backward passes by using the Huchtinson's trick:

$$\mathbb{E}[tr(A)]=\mathbb{E}_{x}[x^\top A x]$$

where, $x$ is a random variable such that $\mathbb{E}[x]=0$ and $\mathbb{E}[xx^\top] = I$. Popular choices are Rademacher and Gaussian Distributions. 

Can you use the Hutchinson's trick to estimate the trace of the input-output Jacobian? Your output should be almost identical to the exact trace for the suggested configuration of the Neural Network. 

In [ ]:
def exact_trace(iNN,x):
    x.requires_grad = True
    y = iNN(x)
    sum = 0
    for i in range(x.shape[1]):
        sum += torch.autograd.grad(y[0,i],x,retain_graph=True)[0][0,i]
    return sum

def approximate_trace(iNN,x):
    
    return None

iNN = InvertibleNN(1000)

r = requests.get("https://web.cs.dal.ca/~sastry/inv_net_1000.pt")
iNN.load_state_dict(torch.load(io.BytesIO(r.content)))

for i in range(10):
    x = torch.FloatTensor(np.random.randn(1,1000))
    print(exact_trace(iNN,x))
    print(approximate_trace(iNN,x))
    print("="*30)

# III. SGD

In this section, we will solve a simple linear regression problem, wherein we require x+y+z = 1. However, a particular application requires all the x,y and z values to be within 0 and 1 and the following code chooses x,y, and z values which are sometimes beyond the said limits. 

Without modifying the loss function, can you modify the below code such that the above-mentioned condition is satisfied? 

Hint: Apply a transformation F to p such that $0\le F(p) \le 1$ 

In [ ]:
p = torch.nn.Parameter(torch.FloatTensor(np.random.randn(3)))
it = 0

while it<=1000:
    loss = 0.5 * (p.sum()-1)**2
    p = p - 0.01*torch.autograd.grad(loss,p)[0]
    p = f.softmax(p,dim=0)
    it += 1
print(p.sum().item(),p.detach().numpy(),loss.item())

1.0 [0.33333334 0.33333334 0.33333334] 0.0
